In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import sys
import requests
import numpy as np
from config import api_key

In [3]:
nasdaq_url = 'https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey=' + api_key
print(nasdaq_url)


https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey=1e66f91abd67cd77b6d1a6265607e908


In [4]:
response = requests.get(nasdaq_url).json()
print(json.dumps(response, indent=4, sort_keys=True))

[
    {
        "cik": "0001261333",
        "dateFirstAdded": "2020-06-22",
        "founded": "2018-04-27",
        "headQuarter": "San Francisco, CALIFORNIA",
        "name": "DocuSign Inc",
        "sector": "Technology",
        "subSector": "Technology",
        "symbol": "DOCU"
    },
    {
        "cik": "0001585521",
        "dateFirstAdded": "2020-04-30",
        "founded": "2019-04-18",
        "headQuarter": "San Jose, CALIFORNIA",
        "name": "Zoom Video Communications Inc",
        "sector": "Communication Services",
        "subSector": "Communication Services",
        "symbol": "ZM"
    },
    {
        "cik": "0001093557",
        "dateFirstAdded": "2020-04-20",
        "founded": "2005-04-14",
        "headQuarter": "San Diego, CALIFORNIA",
        "name": "DexCom Inc",
        "sector": "Healthcare",
        "subSector": "Healthcare",
        "symbol": "DXCM"
    },
    {
        "cik": "0001353283",
        "dateFirstAdded": "2019-12-23",
        "founded": "20

In [5]:
# List of Companies
nasdaqcompanies = []
nasdaqsymbols = []

for company in response:
    nasdaqcompanies.append(company['name'])
    nasdaqsymbols.append(company['symbol'])


In [6]:
data = {'Company': nasdaqsymbols,
        'Q1 Date': ['' for i in range(len(nasdaqcompanies))],
        'Q1 Price': ['' for i in range(len(nasdaqsymbols))], 
        'Q2 Date': ['' for i in range(len(nasdaqsymbols))],
        'Q2 Price': ['' for i in range(len(nasdaqsymbols))], 
        'Q3 Date': ['' for i in range(len(nasdaqsymbols))],
        'Q3 Price': ['' for i in range(len(nasdaqsymbols))],  
       }

nasdaq_df = pd.DataFrame(data=data)
nasdaq_df.head()

,Company,Q1 Date,Q1 Price,Q2 Date,Q2 Price,Q3 Date,Q3 Price
0,DOCU,,,,,,
1,ZM,,,,,,
2,DXCM,,,,,,
3,SPLK,,,,,,
4,SGEN,,,,,,


In [7]:
# Historical Data Collection

for index, row in nasdaq_df.iterrows():
        
    companysymbol = row["Company"]
    date_url = f'https://financialmodelingprep.com/api/v3/ratios/{companysymbol}?period=quarter&limit=3&apikey={api_key}'
    response = requests.get(date_url).json()    
    
    try:  
        
        nasdaq_df.loc[index, "Q1 Date"] = response[2]["date"]
        nasdaq_df.loc[index, "Q2 Date"] = response[1]["date"]
        nasdaq_df.loc[index, "Q3 Date"] = response[0]["date"]
        
    except IndexError:
        print(f"N/A")
        
   

    
print(nasdaq_df)
    
    


    Company     Q1 Date Q1 Price     Q2 Date Q2 Price     Q3 Date Q3 Price
0      DOCU  2020-01-31           2020-04-30           2020-07-31         
1        ZM  2020-01-31           2020-04-30           2020-07-31         
2      DXCM  2019-12-31           2020-03-31           2020-06-30         
3      SPLK  2020-01-31           2020-04-30           2020-07-31         
4      SGEN  2019-12-31           2020-03-31           2020-06-30         
..      ...         ...      ...         ...      ...         ...      ...
98     NVDA  2020-01-26           2020-04-26           2020-07-26         
99     PAYX  2020-02-29           2020-05-31           2020-08-31         
100    KLAC  2019-12-31           2020-03-31           2020-06-30         
101    CSCO  2020-01-25           2020-04-25           2020-07-25         
102    ADSK  2020-01-31           2020-04-30           2020-07-31         

[103 rows x 7 columns]


In [8]:
nasdaq_df.head()

,Company,Q1 Date,Q1 Price,Q2 Date,Q2 Price,Q3 Date,Q3 Price
0,DOCU,2020-01-31,,2020-04-30,,2020-07-31,
1,ZM,2020-01-31,,2020-04-30,,2020-07-31,
2,DXCM,2019-12-31,,2020-03-31,,2020-06-30,
3,SPLK,2020-01-31,,2020-04-30,,2020-07-31,
4,SGEN,2019-12-31,,2020-03-31,,2020-06-30,


In [9]:
base_url = "https://financialmodelingprep.com/api/v3/historical-price-full/"
for index, row in nasdaq_df.iterrows():
    
    companysymbol = row["Company"]
    
    # Quarter 1 Price Response
    q1_date = row["Q1 Date"]
    q1_price_url = f'{base_url}{companysymbol}?from={q1_date}&to={q1_date}&apikey={api_key}'
    response = requests.get(q1_price_url).json() 
    
    
    try:  
        nasdaq_df.loc[index, "Q1 Price"] = response['historical'][0]['close']
        
    except (IndexError, KeyError):
        print(f"N/A")
    
    
    #Quarter 2 Price Response2
    q2_date = row["Q2 Date"]
    q2_price_url = f'{base_url}{companysymbol}?from={q2_date}&to={q2_date}&apikey={api_key}'
    response2 = requests.get(q2_price_url).json() 
    
    
    try:  
        nasdaq_df.loc[index, "Q2 Price"] = response2['historical'][0]['close']
    
    except (IndexError, KeyError):        
            print(f"N/A")
        
    
    #Quarter 3 Price Response3 
    q3_date = row["Q3 Date"]
    q3_price_url = f'{base_url}{companysymbol}?from={q3_date}&to={q3_date}&apikey={api_key}'
    response3 = requests.get(q3_price_url).json()
    
     
    try:  
        nasdaq_df.loc[index, "Q3 Price"] = response3['historical'][0]['close']
    
    except (IndexError, KeyError):
        print(f"N/A")
    
    
   
    

N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A


In [10]:
nasdaq_df.head(10)

,Company,Q1 Date,Q1 Price,Q2 Date,Q2 Price,Q3 Date,Q3 Price
0,DOCU,2020-01-31,78.51,2020-04-30,104.75,2020-07-31,216.83
1,ZM,2020-01-31,76.3,2020-04-30,135.17,2020-07-31,253.91
2,DXCM,2019-12-31,218.74,2020-03-31,269.27,2020-06-30,405.4
3,SPLK,2020-01-31,155.26,2020-04-30,140.36,2020-07-31,209.82
4,SGEN,2019-12-31,114.26,2020-03-31,115.38,2020-06-30,169.92
5,CDW,2019-12-31,141.454,2020-03-31,92.6278,2020-06-30,115.783
6,INTC,2019-12-28,,2020-03-28,,2020-06-27,
7,XEL,2019-12-31,62.6646,2020-03-31,59.9016,2020-06-30,62.5
8,GILD,2019-12-31,63.8124,2020-03-31,74.1004,2020-06-30,76.94
9,TTWO,2019-12-31,122.43,2020-03-31,118.61,2020-06-30,139.57


In [11]:
nasdaq_df.to_csv('../output/nasdaq_historical_prices.csv')